In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import glob
from reprojection import *

In [2]:
files = sorted(glob.glob('/home/ulyanov/data/solo/phi/polar/*'))
print(len(files))
print(files[0], files[-1])

275
/home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250426T000003_V202601302305_0544260501.fits.gz /home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250531T182003_V202601302340_0545310507.fits.gz


In [6]:
nx, ny = 1024, 1024
xc, yc = 511.5, 511.5
Rsun = 512

view_new = View(nx, ny, xc, yc, Rsun, -90, 90, 0) ### North pole view
grid = np.mgrid[:nx,:ny].astype(np.float32)

mean, coverage = np.zeros_like(grid[0]), np.zeros_like(grid[0])

for file in files:
    print(file)

    with fits.open(file) as hdul:
        header = hdul[0].header.copy()
        data = hdul[0].data.copy()

    view = View.from_header(header)

    transform = view_new.to_spherical(correct_mu=True, mu_thr=0.1) - view.to_spherical(correct_mu=True, correct_dr=True, mu_thr=0.1)
    grid_, alpha = transform(grid)
    data = bilinear(data, *grid_) * alpha

    n = (~np.isnan(data)) * (1 / alpha) ** 2
    coverage += np.nan_to_num(n)
    mean += np.nan_to_num((data - mean) * n / coverage)

mean[coverage == 0] = np.nan
coverage[coverage == 0] = np.nan

/home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250426T000003_V202601302305_0544260501.fits.gz
/home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250426T020003_V202601302305_0544260502.fits.gz
/home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250426T040004_V202601302305_0544260503.fits.gz
/home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250426T060004_V202601302305_0544260504.fits.gz
/home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250426T080004_V202601302305_0544260505.fits.gz
/home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250426T100003_V202601302305_0544260506.fits.gz
/home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250426T120004_V202601302305_0544260507.fits.gz
/home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250426T140004_V202601302305_0544260508.fits.gz
/home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250426T160004_V202601302305_0544260509.fits.gz
/home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250426T

In [7]:
def show_polar_data(data, figsize=(10,10), label='', **kwargs):
    fig = plt.figure(figsize=figsize)

    axes_coords = (0.05, 0.05, 0.9, 0.9)

    ax_image = fig.add_axes(axes_coords)
    im = ax_image.imshow(data, origin='lower', **kwargs)

    ax_image.axis('off')

    cax = ax_image.inset_axes((0.8, 0.985, 0.2, 0.015))
    fig.colorbar(im, cax=cax, orientation='horizontal', label=label)

    ax_polar = fig.add_axes(axes_coords, projection = 'polar')
    ax_polar.patch.set_alpha(0)

    lats = np.arange(15,91,15)

    ax_polar.set_rorigin(0)
    ax_polar.set_rticks(np.cos(lats * np.pi / 180))
    ax_polar.set_rlabel_position(-90)
    ax_polar.set_yticklabels([rf'{lat}$\degree$' for lat in lats], color='black')

    ax_polar.grid(True, ls='--', color='black', alpha=0.6)

    return fig, ax_image

In [8]:
show_polar_data(mean, vmin=-20, vmax=20, cmap='seismic')

(<Figure size 1000x1000 with 2 Axes>, <Axes: >)